In [94]:
# Import required libraries
import pandas as pd
import dash
from dash import Dash, html, dcc, callback, Output, Input
import plotly.express as px

In [95]:
# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

In [96]:
spacex_df.head()

,Unnamed: 0,Flight Number,Launch Site,class,Payload Mass (kg),Booster Version,Booster Version Category
0,0,1,CCAFS LC-40,0,0.0,F9 v1.0 B0003,v1.0
1,1,2,CCAFS LC-40,0,0.0,F9 v1.0 B0004,v1.0
2,2,3,CCAFS LC-40,0,525.0,F9 v1.0 B0005,v1.0
3,3,4,CCAFS LC-40,0,500.0,F9 v1.0 B0006,v1.0
4,4,5,CCAFS LC-40,0,677.0,F9 v1.0 B0007,v1.0


In [97]:
# Create a dash application
app = dash.Dash(__name__)


In [98]:
# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                dcc.Dropdown(
                                    id='site-dropdown',
                                    options=[
                                        {'label': 'All Sites', 'value': 'ALL'},
                                        {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                                        {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
                                        {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                                        {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'}
                                    ],
                                        value='ALL',
                                        placeholder='Please choose a launch site',
                                        searchable=True
                                ),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                #dcc.RangeSlider(id='payload-slider',...)
                                dcc.RangeSlider(
                                    id='payload-slider',
                                    min=0,
                                    max=10000,
                                    step=1000,
                                    marks={0:'0', 2500:'2500', 5000:'5000', 7500:'7500', 10000:'10000'},
                                    value=[min_payload, max_payload]),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])


In [99]:
# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@callback(
    Output(component_id='success-pie-chart', component_property='figure'),
    Input(component_id='site-dropdown', component_property='value')
)
def get_pie_chart(entered_site):
    if entered_site == 'ALL':
        data = spacex_df.groupby('Launch Site')[['Launch Site', 'class']].sum(numeric_only=True).reset_index()
        fig = px.pie(
            data, 
            values='class', 
            names='Launch Site',
            title='Shares of success by launch sites')
    else:
        data = spacex_df[spacex_df['Launch Site']==entered_site][['class']].value_counts().reset_index(name='counts')
        fig = px.pie(
                    data, 
                    values='counts', 
                    names=['Failure', 'Success'],
                    title='Proportions of successful launches from site {}'.format(entered_site)
                )
    return fig


In [100]:
# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@callback(
    Output(component_id='success-payload-scatter-chart', component_property='figure'),
    Input(component_id='site-dropdown', component_property='value'),
    Input(component_id='payload-slider', component_property='value')
)

def get_scatter_plot(site, payload):
    if site == 'ALL':
        data = spacex_df[spacex_df['Payload Mass (kg)'] <= payload[1]]
        data = data[data['Payload Mass (kg)'] >= payload[0]]
        fig = px.scatter(data, x='Payload Mass (kg)', y='class', color='Booster Version Category', title='Correlation between success and payload for all sites for range {} - {} kg'.format(payload[0], payload[1]))
    else:
        data = spacex_df[spacex_df['Launch Site'] == site]
        data = data[data['Payload Mass (kg)'] >= payload[0]]
        data = data[data['Payload Mass (kg)'] <= payload[1]]
        fig = px.scatter(data, x='Payload Mass (kg)', y='class', color='Booster Version Category', title='Correlation between success and payload for {} with range {} - {} kg'.format(site, payload[0], payload[1]))
    return fig



In [101]:
spacex_df[spacex_df['Payload Mass (kg)'] >= 4000]

,Unnamed: 0,Flight Number,Launch Site,class,Payload Mass (kg),Booster Version,Booster Version Category
9,9,11,CCAFS LC-40,0,4535.0,F9 v1.1,v1.1
10,10,12,CCAFS LC-40,0,4428.0,F9 v1.1 B1011,v1.1
14,14,16,CCAFS LC-40,0,4159.0,F9 v1.1 B1014,v1.1
16,16,18,CCAFS LC-40,0,4707.0,F9 v1.1 B1016,v1.1
19,19,22,CCAFS LC-40,0,5271.0,F9 FT B1020,FT
21,21,24,CCAFS LC-40,1,4696.0,F9 FT B1022,FT
25,25,28,CCAFS LC-40,1,4600.0,F9 FT B1026,FT
28,28,29,VAFB SLC-4E,1,9600.0,F9 FT B1029.1,FT
29,29,37,VAFB SLC-4E,1,9600.0,F9 FT B1036.1,FT
31,31,42,VAFB SLC-4E,1,9600.0,F9 B4 B1041.1,B4


In [102]:
# Run the app
if __name__ == '__main__':
    app.run_server()